Uyen Huynh

# RESTAURANT BUSINESSES AROUND GEORGE MASON UNIVERSITY IN FAIRFAX, VIRGINIA

This notebook is to present the process of collecting data using Yelp Fusion API and Google Maps API:
1. **Yelp Fusion API**: 
- Business Search Endpoint: return detailed information about businesses located in major cities/towns/CDPs near George Mason Univerisity campus (4400 University Dr, Fairfax, VA 22030).
- Reviews Endpoints: provide review excerpts from users.   
2. **Distance Matrix API**: get the driving distance (km) and driving travel time (minute) from the GMU campus to a given business.

After collecting data, we also perform data cleansing & data transformation so that the data can be populated into the designed database.  

## Table of contents
[A. Yelp Fusion - Business Search](#first)  
[B. Distance Matrix API](#second)  
[C. Yelp Fusion - Reviews](#third)  

## A. Yelp Fusion - Business Search <a name="first"></a>

### Data Collection

At this step, we send requests to Yelp Fusion API to get the business information based on the provided location criteria.  

In [1]:
import requests
import json
import os
import pandas as pd
import numpy as np
import re

In [2]:
api_key = 'abc' # insert your API key

In [3]:
headers = {'Authorization': 'Bearer %s' % api_key}
url = 'https://api.yelp.com/v3/businesses/search'

# current working directory
cwd = os.getcwd()

locations = ['Fairfax VA', 'Burke VA', 'Springfield VA', 'Annandale VA', 'Tysons Corner VA', 'Vienna VA', \
             'Falls Church VA', 'Centreville VA', 'Chantilly VA', 'Idylwood VA', 'North Springfield VA', \
            'Oakton VA', 'West Falls Church VA', 'West Springfield VA', 'Reston VA', 'Washington DC'] 
             
for loc in locations:
    offset = 0
    # for a location, firstly check whether we already collected & saved data in a file
    file_name = loc.lower().replace(' ', '_') + '.json'
    file_path = os.path.join(cwd, 'data', file_name)

    if not os.path.isfile(file_path):
        businesses = []
        for n in range(10):
            params = {'location':loc, 'limit':'50', 'offset': offset}
            response = requests.get(url, params=params, headers=headers)
            result = json.loads(response.text)
            businesses.append(result)
            offset += 50
    
        # save data in a file
        with open(file_path, 'w') as f:
            json.dump(businesses, f, indent = 4)
    
    print('Done for {}'.format(loc))

Done for Fairfax VA
Done for Burke VA
Done for Springfield VA
Done for Annandale VA
Done for Tysons Corner VA
Done for Vienna VA
Done for Falls Church VA
Done for Centreville VA
Done for Chantilly VA
Done for Idylwood VA
Done for North Springfield VA
Done for Oakton VA
Done for West Falls Church VA
Done for West Springfield VA
Done for Reston VA
Done for Washington DC


### Load the raw data into a dictionary

In [4]:
businesses_dict = {}
locations_lower = []
for loc in locations:
    file_name = loc.lower().replace(' ', '_') + '.json'
    file_path = os.path.join(cwd, 'data', file_name)
    location_key = loc.lower().replace(' ', '_')
    locations_lower.append(location_key)
    with open(file_path, 'r') as f:
        businesses_dict[location_key] = json.load(f)

### Create the dataframe *'businesses_df'* to store the raw data

In [5]:
for a in businesses_dict.keys():
    print(a)

fairfax_va
burke_va
springfield_va
annandale_va
tysons_corner_va
vienna_va
falls_church_va
centreville_va
chantilly_va
idylwood_va
north_springfield_va
oakton_va
west_falls_church_va
west_springfield_va
reston_va
washington_dc


In [6]:
# create a dataframe which stores detailed information of all businesses
businesses_df = pd.DataFrame()

for key, value in businesses_dict.items():
    for i in range(len(value)):
        info_50_businesses = value[i]['businesses']
        for info_1_business in info_50_businesses:
            info_1_business_df = pd.DataFrame.from_dict(info_1_business, orient='index').T
            businesses_df = pd.concat([businesses_df, info_1_business_df], axis=0)

businesses_df.reset_index(drop=True, inplace=True)

In [7]:
businesses_df.head(3)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,KSPpsm1Kxxg8Y2sgQKuqSQ,de-clieu-coffee-and-sandwich-fairfax,De Clieu Coffee & Sandwich,https://s3-media3.fl.yelpcdn.com/bphoto/Xsjorj...,False,https://www.yelp.com/biz/de-clieu-coffee-and-s...,845,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 38.845808, 'longitude': -77.305247}",[delivery],$,"{'address1': '10389 Main St', 'address2': 'Ste...",+17035910309,(703) 591-0309,1465.95
1,iKyRzxyEN-C-D6NrEEa7TA,sisters-thai-fairfax,Sisters Thai,https://s3-media1.fl.yelpcdn.com/bphoto/4Rm61G...,False,https://www.yelp.com/biz/sisters-thai-fairfax?...,1114,"[{'alias': 'thai', 'title': 'Thai'}]",4,"{'latitude': 38.84576, 'longitude': -77.30561}","[pickup, delivery]",$$,"{'address1': '4004 University Dr', 'address2':...",+17032679619,(703) 267-9619,1458.46
2,DtU_GcDsnOpOcowgaVgf0Q,sushi-prince-fairfax,Sushi Prince,https://s3-media1.fl.yelpcdn.com/bphoto/m3AgZ_...,False,https://www.yelp.com/biz/sushi-prince-fairfax?...,853,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4,"{'latitude': 38.8452632, 'longitude': -77.3010...","[pickup, delivery]",$$,"{'address1': '10256 Main St', 'address2': '', ...",+17033830588,(703) 383-0588,1757.79


### Data Cleaning & Data Transformation

In [8]:
# check the size of the dataframe
print('The dataframe "businesses_df" has {} rows and {} columns.'.format(businesses_df.shape[0], businesses_df.shape[1]))

The dataframe "businesses_df" has 8846 rows and 16 columns.


In [9]:
# drop the duplicates
businesses_df.drop_duplicates(subset='id', inplace=True, ignore_index=True)
print('After being removed duplicated rows, the "businesses_df" dataframe contains {} rows.'.format(len(businesses_df)))

After being removed duplicated rows, the "businesses_df" dataframe contains 3721 rows.


#### 1. Coordinates

We need to extract **latitude** & **longitude** from **'coordinates'** values. 

In [10]:
yelp_businesses_df = businesses_df.copy()
# extract latitude
yelp_businesses_df['latitude'] = yelp_businesses_df['coordinates'].apply(lambda x:x['latitude'])
# extract longitude
yelp_businesses_df['longitude'] = yelp_businesses_df['coordinates'].apply(lambda x:x['longitude'])

#### 2. Street, City, State, Zipcode

The **location** column contains the full address of each business. Each value is stored as a dictionary, including the following keys:   
- *address1*, *address2*, *address3*: these elements provides the full street address
- *city*
- *state*
- *zipcode*

We will store *street*, *city*, *state* and *zip_code* values in four separate columns. 

In [11]:
# extract street address
def street_address(a):
    if a['address1'] != None and a['address1'] != '':
        address1 = str(a['address1'])
    else:
        address1 = '' 
    
    if a['address2'] != None and a['address2'] != '':
        address2 = ', ' + str(a['address2'])
    else:
        address2 = ''
    
    if a['address3'] != None and a['address3'] != '':
        address3 = ', ' + str(a['address3'])
    else:
        address3 = ''
    
    street = address1 + address2 + address3
    return street

yelp_businesses_df['street'] = yelp_businesses_df['location'].apply(street_address)

# city
yelp_businesses_df['city'] = yelp_businesses_df['location'].apply(lambda x:x['city'])

# state
yelp_businesses_df['state'] = yelp_businesses_df['location'].apply(lambda x:x['state'])

# zip code
yelp_businesses_df['zip_code'] = yelp_businesses_df['location'].apply(lambda x:x['zip_code'])

# display_address
yelp_businesses_df['display_address'] = yelp_businesses_df['location'].apply(lambda x:x['display_address'])

In [12]:
# recheck the business locations returned by Yelp API
yelp_businesses_df['state'].unique()

array(['VA', 'MD', 'DC', 'YOR', 'WV', 'TX', 'OR', 'OK', 'IL'],
      dtype=object)

In [13]:
# remove businesses which are not located in VA or DC
yelp_businesses_df = yelp_businesses_df[(yelp_businesses_df['state'] == 'VA')|(yelp_businesses_df['state'] == 'DC')]

In [15]:
# unique values of the 'city' column
yelp_businesses_df['city'].unique()

array(['Fairfax', 'Vienna', 'Fairfax Station', 'Falls Church', 'Oakton',
       'Centreville', 'Fairfax City', 'Chantilly', 'Annandale',
       'Springfield', 'Dunn Lorring', 'Reston', 'McLean', 'Arlington',
       'Burke', 'Clifton', 'Tysons', 'Herndon', "Tyson's Corner-Vienna",
       'Alexandria', 'Manassas', 'Sterling', 'Merrifield', 'Great Falls',
       'Ashburn', 'Tysons Corner', "Bailey's Crossroads",
       'Washington, DC', 'Mclean', 'Manassas Park', 'Lorton', 'Aldie',
       'West Springfield', 'North Springfield', 'Newington', 'Bristow',
       'Gainesville', 'Mesa Arizona', 'Sully', 'Woodbridge',
       'Fredericksburg', 'South Riding', 'West Falls Church', 'Northern',
       'Culpeper', 'Mount Vernon', 'Kingstowne', 'Baileys Crossroad',
       'Fort Belvoir', 'Lincolnia', 'Baileys Crossroads', 'Seven Corners',
       "Tyson's Corner", 'Mc Lean', 'Falls church', 'Tysons Corne',
       'West McLean', 'Dunn Loring', 'Bluemont', 'Rosslyn', 'Yorkshire',
       'Sully Station',

As we can see from the result above, some cities are not presented in a consistent way, for example, "Bailey's Crossroads" and "Bailey Crossroads" are exactly the same location. Therefore, we need to adjust some values so that each location will be denoted by only one value. 

In [16]:
def city_correct(x):
    if x == 'Fairfax City':
        return 'Fairfax'
    elif x in ("Tyson's Corner-Vienna", "Tysons", "Tysons Corne", "Tyson's Corner"):
        return 'Tysons Corner'
    elif x == 'Washington, DC':
        return 'Washington DC'
    elif x in ("Bailey's Crossroads", 'Baileys Crossroads', 'Baileys Crossroad'):
        return 'Bailey Crossroads'
    elif x == 'Falls church':
        return 'Falls Church'
    elif x in ('Mc Lean', 'Mclean'):
        return 'McLean'
    elif x == 'Sully Station':
        return 'Centreville'
    elif x == 'Dunn Lorring':
        return 'Dunn Loring'
    else:
        return x
    
yelp_businesses_df['city'] = yelp_businesses_df['city'].apply(city_correct)

In [17]:
# create a list of cities/towns/CDPs where the number of businesses located is less than 10
count_loc = yelp_businesses_df['city'].value_counts().reset_index()
remove_loc = list(count_loc[count_loc['city'] < 10]['index'])
# remove these locations from the dataframe 
def city_correct2(x):
    if x in remove_loc:
        return 'remove'
    else:
        return x
yelp_businesses_df['city'] = yelp_businesses_df['city'].apply(city_correct2)
yelp_businesses_df = yelp_businesses_df[yelp_businesses_df['city'] != 'remove']

#### 3. Price

In [19]:
# check the distinct values of the price
yelp_businesses_df['price'].unique()

array(['$', '$$', '$$$', nan, '$$$$'], dtype=object)

We will convert the **price** column from categorical data to numerical data as the following steps:
- \\$: become to 1.
- \\$\\$: become to 2.
- \\$\\$\\$: become to 3.
- \\$\\$\\$\\$: become to 4.

In [20]:
def convert_price(x):
    if x == '$':
        return 1
    elif x == '$$':
        return 2
    elif x == '$$$':
        return 3
    elif x == '$$$$':
        return 4

In [21]:
yelp_businesses_df['price'] = yelp_businesses_df['price'].apply(convert_price)

#### 4. Working with missing values

In this project, our goal is to make suggestions that match with a student's criteria such as distance or restaurant type. Therefore, if a business doesn't have an obvious address (no street address & only show city information), we cannot calculate the approximate distance and commuting time for users to consider. For the next steps, we need to remove businesses which don't have a full address. 

In [22]:
# check which restaurants don't have the full address
yelp_businesses_df[yelp_businesses_df['street'] == '']

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,...,phone,display_phone,distance,latitude,longitude,street,city,state,zip_code,display_address
111,85xkWE5SZQ6H5gSZ8FmytA,sussys-cakes-fairfax-4,Sussy's Cakes,https://s3-media2.fl.yelpcdn.com/bphoto/ppbxP1...,False,https://www.yelp.com/biz/sussys-cakes-fairfax-...,36,"[{'alias': 'customcakes', 'title': 'Custom Cak...",5,"{'latitude': 38.8060646057129, 'longitude': -7...",...,+17035950341,(703) 595-0341,3845.17,38.806065,-77.296906,,Fairfax,VA,22032,"[Fairfax, VA 22032]"
119,x4p7zEKR4X9x3dhDuQRKrA,sugar-defined-fairfax-3,Sugar Defined,https://s3-media2.fl.yelpcdn.com/bphoto/3JTrHk...,False,https://www.yelp.com/biz/sugar-defined-fairfax...,30,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",5,"{'latitude': 38.8268355, 'longitude': -77.2922...",...,+15713547235,(571) 354-7235,3845.17,38.826836,-77.292233,,Fairfax,VA,22032,"[Fairfax, VA 22032]"
204,0eUse5IhFL5CyOHEMhf31Q,sophisticake-fairfax,Sophisticake,https://s3-media4.fl.yelpcdn.com/bphoto/8YDDyt...,False,https://www.yelp.com/biz/sophisticake-fairfax?...,24,"[{'alias': 'cupcakes', 'title': 'Cupcakes'}, {...",5,"{'latitude': 38.80596, 'longitude': -77.29672}",...,+17033628790,(703) 362-8790,3845.17,38.805960,-77.296720,,Fairfax,VA,22032,"[Fairfax, VA 22032]"
212,DdWsb2a3NjMO6Hw7tWE5LA,the-cupcake-delivers-fairfax,The Cupcake Delivers,https://s3-media2.fl.yelpcdn.com/bphoto/kHRBV3...,False,https://www.yelp.com/biz/the-cupcake-delivers-...,54,"[{'alias': 'fooddeliveryservices', 'title': 'F...",4.5,"{'latitude': 38.8756942749023, 'longitude': -7...",...,+15712150307,(571) 215-0307,6711.89,38.875694,-77.383904,,Fairfax,VA,22033,"[Fairfax, VA 22033]"
223,azp40EHJ_RCa62o6t219RQ,artzy-custom-cakes-oakton,Artzy Custom Cakes,https://s3-media3.fl.yelpcdn.com/bphoto/hZVj47...,False,https://www.yelp.com/biz/artzy-custom-cakes-oa...,54,"[{'alias': 'customcakes', 'title': 'Custom Cak...",5,"{'latitude': 38.891056060791, 'longitude': -77...",...,+17038889401,(703) 888-9401,4128.77,38.891056,-77.329781,,Oakton,VA,22124,"[Oakton, VA 22124]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,hxM4fKurGzS5WfB9kMTK3A,phowheels-washington-2,PhoWheels,https://s3-media2.fl.yelpcdn.com/bphoto/aYuybR...,False,https://www.yelp.com/biz/phowheels-washington-...,343,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.5,"{'latitude': 38.90828, 'longitude': -77.06291}",...,,,3758.21,38.908280,-77.062910,,Washington DC,DC,20007,"[Washington, DC 20007]"
3496,-i2jExohDYLUnuIE62AB3g,basil-thyme-washington,Basil Thyme,https://s3-media2.fl.yelpcdn.com/bphoto/uZK9HP...,False,https://www.yelp.com/biz/basil-thyme-washingto...,298,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 38.87785, 'longitude': -77.01568}",...,+18555887282,(855) 588-7282,4039.53,38.877850,-77.015680,,Washington DC,DC,20024,"[Washington, DC 20024]"
3550,-DO9zy9e9B4bx_g83F0kkw,pho-junkies-washington,Pho Junkies,https://s3-media1.fl.yelpcdn.com/bphoto/CTiYaX...,False,https://www.yelp.com/biz/pho-junkies-washingto...,169,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.5,"{'latitude': 38.87742, 'longitude': -77.01632}",...,+12026435401,(202) 643-5401,4039.53,38.877420,-77.016320,,Washington DC,DC,20024,"[Washington, DC 20024]"
3689,x_7P7SNHmPKpLVnlY_IvPQ,far-east-taco-grille-truck-washington,Far East Taco Grille - Truck,https://s3-media1.fl.yelpcdn.com/bphoto/CpoQNs...,False,https://www.yelp.com/biz/far-east-taco-grille-...,222,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4,"{'latitude': 38.919849395752, 'longitude': -77...",...,+12026014346,(202) 601-4346,1001.21,38.919849,-77.037216,,Washington DC,DC,20009,"[Washington, DC 20009]"


In [23]:
# remove businesses without a full address
yelp_businesses_df = yelp_businesses_df[yelp_businesses_df['street'] != '']

In [24]:
# check null values
yelp_businesses_df.isnull().sum()

id                   0
alias                0
name                 0
image_url            0
is_closed            0
url                  0
review_count         0
categories           0
rating               0
coordinates          0
transactions         0
price              706
location             0
phone                0
display_phone        0
distance             0
latitude             1
longitude            1
street               0
city                 0
state                0
zip_code             0
display_address      0
dtype: int64

In [25]:
# examine the businesses which don't have null values for latitude and longitude
yelp_businesses_df[yelp_businesses_df['latitude'].isnull()|yelp_businesses_df['longitude'].isnull()]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,...,phone,display_phone,distance,latitude,longitude,street,city,state,zip_code,display_address
2602,iF3hWTz1ru3Q-chtybJTUg,community-event-yelps-donut-social-iii-arlington,Community Event: Yelp's Donut Social III,https://s3-media3.fl.yelpcdn.com/bphoto/0EmfYX...,False,https://www.yelp.com/biz/community-event-yelps...,9,"[{'alias': 'yelpevents', 'title': 'Yelp Events'}]",5,"{'latitude': None, 'longitude': None}",...,,,3273.78,NaN,NaN,"2511 N Harrison St, Duck Donuts",Arlington,VA,22207,"[2511 N Harrison St, Duck Donuts, Arlington, V..."


The Yelp API returns the 'None' values for this business's coordinate. However, we have a detailed address, so we are able to manually find the coordinate (via [link](https://www.businessyab.com/explore/united_states/virginia/arlington_county/arlington/yorktown/north_harrison_street/2511/duck_donuts.html)) & update the data.

In [26]:
yelp_businesses_df.loc[2602,'latitude'] = 38.8977073
yelp_businesses_df.loc[2602,'longitude'] = -77.1395893

In [28]:
len(yelp_businesses_df)

3527

## B. Distance Matrix API <a name='second'></a>

Based on the coordinates of all businesses that we collected from Yelp Fusion API, we can use The Distance Matrix API to get the distance and time travelling between **the origin - the business** and **the destination - GMU Campus** (4400 University Dr, Fairfax, VA 22030). 

### Data collection

In [29]:
import googlemaps
from itertools import tee

In [30]:
google_api_key = "abcd" # insert your API key

In [31]:
gmaps = googlemaps.Client(key=google_api_key)

Firstly, we extract the distance & time travelling from GMU campus to the business with the mode 'driving'.

In [32]:
destination = "4400 University Dr, Fairfax, VA 22030"
gmaps_data_path = os.path.join(cwd, 'data', 'gmaps_distances.json')

# check whether we already collected the data
if os.path.exists(gmaps_data_path):
    with open(gmaps_data_path, 'r') as f:
        gmaps_distance_dict = json.load(f)
else:
    gmaps_distance_dict = {}

for i in range(len(yelp_businesses_df)):
    business_id = yelp_businesses_df.iloc[i]['id']
    latitude = yelp_businesses_df.iloc[i]['latitude']
    longitude = yelp_businesses_df.iloc[i]['longitude']
    origins = (latitude, longitude)
    # send a request
    if business_id not in gmaps_distance_dict:
        result = gmaps.distance_matrix(origins=origins, destinations=destination, mode='driving')
        # save the result into the dictionary 'gmaps_distance_dict'
        gmaps_distance_dict[business_id] = result
    
    if (i+1) % 100 == 0:
        print(f'Done {i+1} businesses.')
    
with open(gmaps_data_path, 'w') as f:
    json.dump(gmaps_distance_dict, f, indent=4)

Done 100 businesses.
Done 200 businesses.
Done 300 businesses.
Done 400 businesses.
Done 500 businesses.
Done 600 businesses.
Done 700 businesses.
Done 800 businesses.
Done 900 businesses.
Done 1000 businesses.
Done 1100 businesses.
Done 1200 businesses.
Done 1300 businesses.
Done 1400 businesses.
Done 1500 businesses.
Done 1600 businesses.
Done 1700 businesses.
Done 1800 businesses.
Done 1900 businesses.
Done 2000 businesses.
Done 2100 businesses.
Done 2200 businesses.
Done 2300 businesses.
Done 2400 businesses.
Done 2500 businesses.
Done 2600 businesses.
Done 2700 businesses.
Done 2800 businesses.
Done 2900 businesses.
Done 3000 businesses.
Done 3100 businesses.
Done 3200 businesses.
Done 3300 businesses.
Done 3400 businesses.
Done 3500 businesses.


### Load the raw data into a dataframe

In [84]:
# load the data
with open(gmaps_data_path, 'r') as f:
    gmaps_distance_dict = json.load(f)

In [85]:
# create a dataframe
business_id_lst = []
distance_lst = []
duration_lst = []

for key, value in gmaps_distance_dict.items():
    business_id = key
    business_id_lst.append(business_id)
    
    # if the API call returns 'no results', we will store the distance & duration values of -1 
    if 'distance' in value['rows'][0]['elements'][0]:
        distance = value['rows'][0]['elements'][0]['distance']['value']
    else:
        distance = -1
    distance_lst.append(distance)
    
    if 'distance' in value['rows'][0]['elements'][0]:
        duration = value['rows'][0]['elements'][0]['duration']['value']
    else:
        duration = -1
    duration_lst.append(duration)
    
ggmaps_df = pd.DataFrame(np.array([business_id_lst, distance_lst, duration_lst])).T
ggmaps_df.columns = ['business_id', 'driving_distance', 'driving_duration']

In [86]:
ggmaps_df.tail()

,business_id,driving_distance,driving_duration
3522,hZ6Bk6thWcFGNRIYwH076A,35166,2185
3523,VDezBOqn852wfl_cRiERsg,34736,1876
3524,3uandIdocQdhEryIZEIF9Q,35986,1953
3525,odvIEE5vX2rb7bvF0B2EOA,35323,2157
3526,eqHzWyJ47ysq9S7m_stKuA,38706,2591


### Data Cleaning & Data Transformation

In [87]:
# check which business_id has no result for distance & duration
condition = (ggmaps_df['driving_distance'] == -1)|(ggmaps_df['driving_duration'] == -1)
ggmaps_no_results = ggmaps_df[condition].copy()

print('The number of businesses that the Distance Matrix API cannot return the distance & duration' \
      'from their locations to GMU campus: {}'.format(len(ggmaps_no_results))) 

The number of businesses that the Distance Matrix API cannot return the distance & durationfrom their locations to GMU campus: 0


In [88]:
ggmaps_df.tail()

,business_id,driving_distance,driving_duration
3522,hZ6Bk6thWcFGNRIYwH076A,35166,2185
3523,VDezBOqn852wfl_cRiERsg,34736,1876
3524,3uandIdocQdhEryIZEIF9Q,35986,1953
3525,odvIEE5vX2rb7bvF0B2EOA,35323,2157
3526,eqHzWyJ47ysq9S7m_stKuA,38706,2591


### Create dataframes which correspond to relations in the database: 'business', 'transaction', 'category', 'review'. 

At the next steps, we will use the data collected above to create dataframes which correspond to relations in our database. 
- **business** (business_id: string, business_name: string, business_url: string, street: string, city: string, state: string, zip_code: string, phone: string, latitude: number, longitude: number, business_rating: number, is_closed: boolean, price: number, review_count: number, driving_distance: number, driving_duration: number).

- **transaction** (transaction_id: string, transaction_name: string).

- **category** (category_id: string, category_name: string).

- **transaction_business** (business_id: references business, transaction_id: references transaction).

- **category_business** (business_id: references business, category_id: references category).

#### a. business

In [89]:
# copy the columns from 'yelp_businesses_df' which is created by the data from Yelp
business_df1 = yelp_businesses_df[['id', 'name', 'url', 'street', 'city', 'state', 'zip_code', 'phone', 'latitude',
       'longitude', 'rating', 'is_closed', 'price', 'review_count']].copy()
# rename some columns
business_df1.rename({'id':'business_id', 'name':'business_name', 'url':'business_url', 
                    'rating':'business_rating'}, axis=1, inplace=True)

In [90]:
# copy 'driving_distance' & 'driving_duration' columns from 'ggmaps_df',  
# which is created by the data from Google Maps - Distance Matrix API
business_df2 = ggmaps_df[['business_id', 'driving_distance', 'driving_duration']].copy()

To create the **business_df** dataframe which saves all data needed for the **business** table in our database, we will merge the **business_df1** & **business_df2** above by the common value *business_id*.

In [91]:
business_df = pd.merge(business_df1, business_df2, how='inner', on='business_id')

In [92]:
# check the data types
business_df.dtypes

business_id          object
business_name        object
business_url         object
street               object
city                 object
state                object
zip_code             object
phone                object
latitude            float64
longitude           float64
business_rating      object
is_closed            object
price               float64
review_count         object
driving_distance     object
driving_duration     object
dtype: object

In [93]:
# change the data types of some columns
business_df['business_rating'] = business_df['business_rating'].astype(float)
business_df['review_count'] = business_df['review_count'].astype(int)
business_df['driving_distance'] = business_df['driving_distance'].astype(float)
business_df['driving_duration'] = business_df['driving_duration'].astype(float)

# convert the unit measure of driving_distance from m to km 
business_df['driving_distance'] = round(business_df['driving_distance']/1000, 2)
# convert the unit measure of driving_duration from second to minute
business_df['driving_duration'] = round(business_df['driving_duration']/60, 2)

In [94]:
business_df['driving_distance'].describe()

count    3527.000000
mean       19.337556
std        19.204514
min         0.040000
25%        11.770000
50%        17.910000
75%        24.970000
max      1008.420000
Name: driving_distance, dtype: float64

In [95]:
# recheck outliers 
distance_outlier = business_df['driving_distance'].max()
business_df[business_df['driving_distance'] == distance_outlier]

,business_id,business_name,business_url,street,city,state,zip_code,phone,latitude,longitude,business_rating,is_closed,price,review_count,driving_distance,driving_duration
2459,iF3hWTz1ru3Q-chtybJTUg,Community Event: Yelp's Donut Social III,https://www.yelp.com/biz/community-event-yelps...,"2511 N Harrison St, Duck Donuts",Arlington,VA,22207,,38.897707,-77.139589,5.0,False,NaN,9,1008.42,560.38


It seems that there is an error for the distance calculation for this business. With this address, Google maps returns: 
- driving distance: 22.05 km
- driving duration: 24 minutes

In [96]:
# update the right result
business_df.iloc[2459, 14] = 22.05
business_df.iloc[2459, 15] = 24.00

In [133]:
print('The business_df contains {} columns and {} rows.'.format(business_df.shape[1], business_df.shape[0]))
business_df.head(3)

The business_df contains 16 columns and 3527 rows.


,business_id,business_name,business_url,street,city,state,zip_code,phone,latitude,longitude,business_rating,is_closed,price,review_count,driving_distance,driving_duration
0,KSPpsm1Kxxg8Y2sgQKuqSQ,De Clieu Coffee & Sandwich,https://www.yelp.com/biz/de-clieu-coffee-and-s...,"10389 Main St, Ste B",Fairfax,VA,22030,+17035910309,38.845808,-77.305247,4.5,False,1.0,845,1.81,4.05
1,iKyRzxyEN-C-D6NrEEa7TA,Sisters Thai,https://www.yelp.com/biz/sisters-thai-fairfax?...,4004 University Dr,Fairfax,VA,22030,+17032679619,38.845760,-77.305610,4.0,False,2.0,1114,1.80,4.37
2,DtU_GcDsnOpOcowgaVgf0Q,Sushi Prince,https://www.yelp.com/biz/sushi-prince-fairfax?...,10256 Main St,Fairfax,VA,22030,+17033830588,38.845263,-77.301053,4.0,False,2.0,853,2.15,6.72


#### b. transaction & transaction_business

The **transactions** column shows a list of transactions that a business is registered for. The guideline on [Yelp](https://www.yelp.com/developers/documentation/v3/business_search) shows that there are three types of Yelp transactions: *pickup*, *delivery*, *restaurant_reservation*.

We will extract the distinct values of the **transactions** column to make sure that there are only three types. 

In [98]:
# check the unique values of the 'transactions' column
yelp_businesses_df['transactions'].apply(lambda x:", ".join(x)).unique()

array(['delivery', 'pickup, delivery', 'delivery, pickup',
       'restaurant_reservation, delivery', '',
       'delivery, restaurant_reservation', 'pickup',
       'restaurant_reservation, delivery, pickup',
       'delivery, restaurant_reservation, pickup',
       'restaurant_reservation, pickup, delivery',
       'pickup, delivery, restaurant_reservation',
       'pickup, restaurant_reservation, delivery',
       'restaurant_reservation', 'restaurant_reservation, pickup'],
      dtype=object)

It can be seen that the list of Yelp transactions contains only 3 values: *pickup*, *delivery*, *restaurant_reservation*.


The next step is to create the **transaction_df** dataframe with two columns: *transaction_id*, *transaction_name*.

In [99]:
# create the 'transaction_df' dataframe
transaction_df = pd.DataFrame(np.array([list(range(1,4)), ['pickup', 'delivery', 'restaurant_reservation']])).T
transaction_df.columns = ['transaction_id', 'transaction_name']
# set the data type
transaction_df['transaction_id'] = transaction_df['transaction_id'].astype(str)

In [100]:
transaction_df.head()

,transaction_id,transaction_name
0,1,pickup
1,2,delivery
2,3,restaurant_reservation


Then, we need to generate the **transaction_business_df** dataframe, including *business_id* and *transaction_id*.

In [101]:
business_ids = []
transaction_ids = []

for i in range(len(yelp_businesses_df)):
    business_id = yelp_businesses_df.iloc[i]['id']
    business_tran = yelp_businesses_df.iloc[i]['transactions']
    if business_tran != '':
        for element in business_tran:
            if element == 'pickup':
                transaction_ids.append(1)
            elif element == 'delivery':
                transaction_ids.append(2)
            elif element == 'restaurant_reservation':
                transaction_ids.append(3)
            business_ids.append(business_id)
            
# create the dataframe 
transaction_business_df = pd.DataFrame(np.array([business_ids, transaction_ids])).T
transaction_business_df.columns = ['business_id', 'transaction_id']

# set the data type
transaction_business_df['transaction_id'] = transaction_business_df['transaction_id'].astype(str)

In [102]:
print('The transaction_business_df contains {} columns and {} rows.'\
      .format(transaction_business_df.shape[1], transaction_business_df.shape[0]))
transaction_business_df.head(3)

The transaction_business_df contains 2 columns and 4378 rows.


,business_id,transaction_id
0,KSPpsm1Kxxg8Y2sgQKuqSQ,2
1,iKyRzxyEN-C-D6NrEEa7TA,1
2,iKyRzxyEN-C-D6NrEEa7TA,2


#### c. category & category_business

The **category_df** dataframe contains 2 columns: *category_id*, *category_name*.

In [103]:
# 'category_name': extract all the unique categories
all_categories = set()
for i in range(len(yelp_businesses_df)):
    categories = yelp_businesses_df.iloc[i]['categories']
    for element in categories:
        category = element['title']
        all_categories.add(category)

In [104]:
print('The number of distinct categories is: {}.'.format(len(all_categories)))

The number of distinct categories is: 261.


In [105]:
# create the 'category_id' column
category_ids = list(range(1, len(all_categories)+1))
# create the 'category_df' dataframe
category_df = pd.DataFrame([category_ids, all_categories]).T
category_df.columns = ['category_id', 'category_name']
# set the data type
category_df['category_id'] = category_df['category_id'].astype(str)

In [145]:
category_df.tail()

,category_id,category_name
256,257,Patisserie/Cake Shop
257,258,Butcher
258,259,DJs
259,260,Arabian
260,261,Steakhouses


Furthermore, we also need to create the '**category_business_df**' dataframe to describe the categories of each business. The dataframe has two columns: *business_id*, *category_id*.

In [107]:
business_ids = []
category_names = []

# for each row, we extract the id & the categories of each business
for i in range(len(yelp_businesses_df)):
    # extract the id
    business_id = yelp_businesses_df.iloc[i]['id']
    # extract the list of categories
    business_cat = yelp_businesses_df.iloc[i]['categories']
    # extract the atomic value 
    for element in business_cat:
        category = element['title']
        business_ids.append(business_id)
        category_names.append(category)
        
# create the 'business_categoryname_df' dataframe
business_categoryname_df = pd.DataFrame([business_ids, category_names]).T
business_categoryname_df.columns = ['business_id', 'category_name']

# merge 'business_categoryname_df' & 'category_df' 
# so that we can match the business_id with its corresponding category_id
category_business_df = pd.merge(business_categoryname_df, category_df, on='category_name', how='left')

# the 'category_business_df' dataframe only needs two columns business_id & category_id,
# so we will drop the 'category_name' column. 
category_business_df.drop('category_name', axis=1, inplace=True)

In [108]:
print('The category_business_df contains {} columns and {} rows.'\
      .format(category_business_df.shape[1], category_business_df.shape[0]))
category_business_df.head(3)

The category_business_df contains 2 columns and 7286 rows.


,business_id,category_id
0,KSPpsm1Kxxg8Y2sgQKuqSQ,76
1,KSPpsm1Kxxg8Y2sgQKuqSQ,142
2,KSPpsm1Kxxg8Y2sgQKuqSQ,113


## C. Yelp Fusion - Reviews <a name='third'></a>

### Data collection

In [109]:
url_review = 'https://api.yelp.com/v3/businesses/KSPpsm1Kxxg8Y2sgQKuqSQ/reviews'

In [110]:
reviews_data_path = os.path.join(cwd, 'data', 'all_review_results.json')

if os.path.exists(reviews_data_path):
    with open(reviews_data_path, 'r') as f:
        all_review_results = json.load(f)
else:
    all_review_results = {}

In [111]:
business_ids = list(yelp_businesses_df['id'].copy())

for i, business_id in enumerate(business_ids):
    # check whether we already collected the reviews of the business or not
    # if not, we will call the API for this business_id
    if business_id not in all_review_results:
        url_review = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(business_id)
        response = requests.get(url_review, headers=headers)
        review_result = json.loads(response.text)
    
        # if the response is an error or there are no reviews for this business, 
        # we will save the result as an empty list.
        if 'reviews' in review_result:
            reviews = review_result['reviews']
            all_review_results[business_id] = reviews
        else:
            all_review_results[business_id] = []
        
    if (i+1) % 100 == 0:
        print(f"Done {i+1} businesses")

Done 100 businesses
Done 200 businesses
Done 300 businesses
Done 400 businesses
Done 500 businesses
Done 600 businesses
Done 700 businesses
Done 800 businesses
Done 900 businesses
Done 1000 businesses
Done 1100 businesses
Done 1200 businesses
Done 1300 businesses
Done 1400 businesses
Done 1500 businesses
Done 1600 businesses
Done 1700 businesses
Done 1800 businesses
Done 1900 businesses
Done 2000 businesses
Done 2100 businesses
Done 2200 businesses
Done 2300 businesses
Done 2400 businesses
Done 2500 businesses
Done 2600 businesses
Done 2700 businesses
Done 2800 businesses
Done 2900 businesses
Done 3000 businesses
Done 3100 businesses
Done 3200 businesses
Done 3300 businesses
Done 3400 businesses
Done 3500 businesses


In [112]:
# save the data in a file
with open(reviews_data_path, 'w') as f:
    json.dump(all_review_results, f, indent = 4)

### Load the raw data into a dataframe

In [113]:
with open(reviews_data_path, 'r') as f:
    all_review_results = json.load(f)

In [114]:
all_review_df = pd.DataFrame()

for business_id, reviews in all_review_results.items():
    if reviews != []:
        for review in reviews:
            review_details = pd.DataFrame.from_dict(review, orient='index').T
            review_details['business_id'] = business_id
            all_review_df = pd.concat([all_review_df, review_details], axis=0)

# change the column name from 'id' to 'review_id'
all_review_df.rename({'id':'review_id', 'rating':'review_rating'}, axis=1, inplace=True)

In [115]:
all_review_df['user_id'] = all_review_df['user'].apply(lambda x: x['id'])
all_review_df['user_profile_url'] = all_review_df['user'].apply(lambda x: x['profile_url'])
all_review_df['user_image_url'] = all_review_df['user'].apply(lambda x: x['image_url'])
all_review_df['user_name'] = all_review_df['user'].apply(lambda x: x['name'])

In [116]:
all_review_df.reset_index(inplace=True, drop=True)
all_review_df.tail(3)

,review_id,url,text,review_rating,time_created,user,business_id,user_id,user_profile_url,user_image_url,user_name
10260,CbMif9x2b66m39lq6H6nFw,https://www.yelp.com/biz/makan-washington?adju...,Stopped by because a friend recommended this p...,5,2021-06-23 19:14:06,"{'id': 'XyjakaDxNLDtm1AYSTXHjQ', 'profile_url'...",eqHzWyJ47ysq9S7m_stKuA,XyjakaDxNLDtm1AYSTXHjQ,https://www.yelp.com/user_details?userid=Xyjak...,None,Ann K.
10261,TagcZMbXOYipKTOV4JzOGQ,https://www.yelp.com/biz/makan-washington?adju...,I've had the pleasure of experiencing Makan fr...,5,2021-07-20 16:50:59,"{'id': 'ts7DQGHG_2zauWTNY6ZByA', 'profile_url'...",eqHzWyJ47ysq9S7m_stKuA,ts7DQGHG_2zauWTNY6ZByA,https://www.yelp.com/user_details?userid=ts7DQ...,https://s3-media4.fl.yelpcdn.com/photo/Rp0cvWE...,Liz K.
10262,LuJ5yhMV0pNzFz-aiQKSTA,https://www.yelp.com/biz/makan-washington?adju...,Really great neighborhood spot with authentic ...,4,2021-07-10 10:10:50,"{'id': 'ZgEnGhJTg7mvLY2a5fYkNg', 'profile_url'...",eqHzWyJ47ysq9S7m_stKuA,ZgEnGhJTg7mvLY2a5fYkNg,https://www.yelp.com/user_details?userid=ZgEnG...,https://s3-media2.fl.yelpcdn.com/photo/pETj1Ts...,Philippe D.


In [117]:
# strip unnecessary white spaces in 'text' values
def text_clean(a):
    new_a = re.sub(r"\s{2,}", " ", a)
    new_a = re.sub(r"\n+", " ", new_a)
    return new_a

all_review_df['text'] = all_review_df['text'].apply(text_clean)

### Create dataframes which correspond to relations in the database: 'review' & 'user'

### a. review

The dataframe **review_df** has 6 columns: *review_id*, *review_rating*, *text*, *time_created*, *business_id*, *user_id*. 

In [118]:
review_df = all_review_df[['review_id', 'review_rating', 'text', 
                           'time_created', 'business_id', 'user_id']].copy().drop_duplicates(subset='review_id')
review_df.reset_index(inplace=True, drop=True)

In [119]:
# check the data types
review_df.dtypes

review_id        object
review_rating    object
text             object
time_created     object
business_id      object
user_id          object
dtype: object

In [120]:
review_df['review_rating'] = review_df['review_rating'].astype(float)

In [121]:
print('The review_df contains {} columns and {} rows.'.format(review_df.shape[1], review_df.shape[0]))
review_df.head(3)

The review_df contains 6 columns and 10263 rows.


,review_id,review_rating,text,time_created,business_id,user_id
0,Gktz5xcIUjGY9Bm5z6Bwgw,5.0,DeClieu is a warm and inviting place as soon a...,2021-06-12 23:51:01,KSPpsm1Kxxg8Y2sgQKuqSQ,WVULRedHN-yKI70Qg79vRQ
1,4fxWmyeYo7GUk73crEjRSg,5.0,Went here with my gf and her family for a morn...,2021-05-30 09:27:36,KSPpsm1Kxxg8Y2sgQKuqSQ,8kWiHzJBgv1CFRTgZwWgaQ
2,Kj1WM0NAj8DeeKPgazPR2Q,4.0,What a great little place to stop in during my...,2021-06-10 18:53:39,KSPpsm1Kxxg8Y2sgQKuqSQ,B9J6L6fVSrJ3cT8lif3hkA


### b. user

The **user_df** dataframe needs to contain 3 columns: *user_id*, *user_name*, *user_profile_url*, *user_image_url*.

In [122]:
user_df = all_review_df[['user_id', 'user_name', 'user_profile_url', 'user_image_url']].copy().drop_duplicates(subset='user_id')
user_df.reset_index(inplace=True, drop=True)

In [123]:
print('The user_df contains {} columns and {} rows.'.format(user_df.shape[1], user_df.shape[0]))
user_df.head(3)

The user_df contains 4 columns and 6549 rows.


,user_id,user_name,user_profile_url,user_image_url
0,WVULRedHN-yKI70Qg79vRQ,Brian L.,https://www.yelp.com/user_details?userid=WVULR...,None
1,8kWiHzJBgv1CFRTgZwWgaQ,Jivak S.,https://www.yelp.com/user_details?userid=8kWiH...,https://s3-media4.fl.yelpcdn.com/photo/51DKHwA...
2,B9J6L6fVSrJ3cT8lif3hkA,Laura C.,https://www.yelp.com/user_details?userid=B9J6L...,https://s3-media2.fl.yelpcdn.com/photo/sX2mRrB...


## D. Export dataframes to CSV files

In [124]:
business_df.to_csv(os.path.join(cwd,'data_csv', 'businesses.csv'), sep= '|', index=False)
transaction_df.to_csv(os.path.join(cwd,'data_csv', 'transactions.csv'), sep= '|', index=False)
category_df.to_csv(os.path.join(cwd,'data_csv', 'categories.csv'), sep= '|', index=False)
review_df.to_csv(os.path.join(cwd,'data_csv', 'reviews.csv'), sep= '|', index=False)
user_df.to_csv(os.path.join(cwd,'data_csv', 'users.csv'), sep= '|', index=False)
transaction_business_df.to_csv(os.path.join(cwd,'data_csv', 'transaction_business.csv'), sep= '|', index=False)
category_business_df.to_csv(os.path.join(cwd,'data_csv', 'category_business.csv'), sep= '|', index=False)

## E. Export files to store SQL queries

We will export script files that store 'insert SQL queries' so that we can populate data into the database. 

In [125]:
def nan_or_none(a):
    return (a == None or a != a)

def insert_query(dataframe, table_name):
    with open(os.path.join(cwd, 'insert_query', (table_name+'.sql')), 'w') as f:
        query1 = "INSERT INTO {} {}\n".format(table_name, tuple(dataframe.columns)).replace("'", "")
        for r in range(len(dataframe)):
            row = dataframe.iloc[r]
            all_values = ''
            for c in range(len(row)):
                value = row[c]
                if c != len(row) - 1:
                    if nan_or_none(value):
                        all_values += 'NULL, '
                    else:
                        if type(value) is str:
                            value = value.replace("'", "''")
                            all_values += "'" + value + "', " 
                        else:
                            all_values += str(value) + ', '
                else:
                    if nan_or_none(value):
                        all_values += 'NULL'
                    else:
                        if type(value) is str:
                            value = value.replace("'", "''")
                            all_values += "'" + value + "'"
                        else:
                            all_values += str(value)
            query2 = "VALUES ({});\n".format(all_values)
            query = query1 + query2
            f.write(query)
        return 'Done for the "{}" table'.format(table_name)

In [126]:
print(insert_query(business_df, 'businesses'))

Done for the "businesses" table


In [127]:
print(insert_query(user_df, 'users'))

Done for the "users" table


In [128]:
print(insert_query(review_df, 'reviews'))

Done for the "reviews" table


In [129]:
print(insert_query(transaction_business_df, 'transaction_business'))

Done for the "transaction_business" table


In [130]:
print(insert_query(category_business_df, 'category_business'))

Done for the "category_business" table


In [131]:
print(insert_query(category_df, 'categories'))

Done for the "categories" table


In [132]:
print(insert_query(transaction_df, 'transactions'))

Done for the "transactions" table
